In [1]:
import augur
from augur.piper_reader import PiperMail
from sqlalchemy.ext.declarative import declarative_base
import json
import pandas as pd
from perceval.backends.core.pipermail import Pipermail,PipermailList
from perceval.backends.core.mbox import MBox
import perceval
import os, os.path
import sqlalchemy as s
from sqlalchemy.orm import scoped_session, sessionmaker
import datetime
import numpy as np
from dateutil.parser import parse

## Load:
First loads the users data from 'augur.config.json' so will take the Database information (e.g. name of database, port of database). Then connects to the database using augur.App.ghtorrentplus() and also loads the piper_reader and loads the path to the list of mailing lists 'runtine/mailing_lists.csv'

In [2]:
if("notebooks" not in os.getcwd()):
    os.chdir("notebooks")
augurApp = augur.Application('../augur.config.json')
connect,list1,path = augurApp.github_issues()
token = list1[5]
DB_STR = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format(
            list1[0], list1[1], list1[2],\
            list1[3], list1[4]
        )
db = s.create_engine(DB_STR)
Piper, path= augurApp.piper()

## Connect:

Queries what tables are in the database and determines if 'mail_lists' is there if it is 'mail_lists' is set as 'True' if it isn't 'mail_lists' is set at 'False'. Then we determine what mailing lists are in 'mailing_list_jobs' and we determine how many rows are in it. If 'mailing_list_jobs' is not in the Database it is created and the column 'augurlistID' is set as the primary key. We then add a connection to 'mailing_list_jobs' so that we can change the column 'last_message_date' if new messages were downloaded for a mailing list.

In [3]:
table_names = s.inspect(db).get_table_names()
print(table_names)
val = False
mail_lists = True
numb = 0
if("mail_lists" not in table_names):
    mail_lists = False
if("mailing_list_jobs" in table_names):
    lists_createdSQL = s.sql.text("""SELECT project FROM mailing_list_jobs""")
    df1 = pd.read_sql(lists_createdSQL, db)
    print(df1)
    val = True
    val = db.engine.execute("""SELECT augurlistID FROM 
                                   mailing_list_jobs
                                   ORDER BY augurlistID DESC LIMIT 1""")
    for i in val:
        numb = i['augurlistID']
    
else:
    columns2 = "augurlistID","backend_name","mailing_list_url","project","last_message_date"
    df_mail_list = pd.DataFrame(columns=columns2)
    df_mail_list.to_sql(name="mailing_list_jobs",con=db,if_exists='replace',index=False,
                        dtype={'augurlistID': s.types.Integer(),
                               'backend_name': s.types.VARCHAR(length=300),
                               'mailing_list_url': s.types.VARCHAR(length=300),
                               'project': s.types.VARCHAR(length=300),
                               'last_message_date': s.types.DateTime()
                        })
    lists_createdSQL = s.sql.text("""SELECT project FROM mailing_list_jobs""")
    df1 = pd.read_sql(lists_createdSQL, db)
    db.execute("ALTER TABLE mailing_list_jobs ADD PRIMARY KEY (augurlistID)")
    print("Created Table")
Base = declarative_base(db)
class table(Base):
    __tablename__ = 'mailing_list_jobs'
    __table_args__={'autoload':True}
    
Session = sessionmaker(bind=db)
session = Session()
res = session.query(table).all()

['github_issues', 'github_issues_2', 'github_issues_repo_jobs', 'github_issues_sentiment_scores', 'github_issues_sentiment_scores_senticr', 'github_pull_request_repo_jobs', 'github_pull_requests', 'github_pull_requests_2', 'github_pull_requests_sentiment_scores', 'github_pull_requests_sentiment_scores_2', 'github_pull_requests_sentiment_scores_senticr', 'issue_response_time', 'issue_response_time2', 'issue_response_time_3', 'mail_lists_sentiment_scores', 'mail_lists_sentiment_scores_senticr']
Created Table


## Sifting through messages:

The function 'write_message' is used to fetch the messages from the downloaded MBox's. The messages downloaded can either be from the last message was downloaded when the jupyter notebook was downloaded previously or if it's the first time all the messages are downloaded. The function 'write_message' then determines which message contains the full message thread by looking to see if the 'ID' for the message is stored in 'References'. Only if the next message downloaded does not reference the previous message will the message be added to dictionary and if a certain amount of messages are stored it is added to the table 'mail_lists' using piper_reader.

In [4]:
def write_message(table,repo,type_archive,mail_check,pos,db,res,session,archives,numb,mail_lists,time=None):
    thread = None
    store = None
    k = 0
    di = {}
    if(time is None):
        time = Piper.convert_date("Thu, 1 Dec 2010 20:37:11 +0000")
        #time = Piper.convert_date("Thu, 1 Aug 2012 20:37:11 +0000")
    for message in repo.fetch(from_date=time):
        if(type_archive == 'not_new'):
            mess_check = Piper.convert_date(message['data']['Date'])
        if(type_archive == 'not_new' and mess_check <= time ):
            continue            
        elif(type_archive == 'not_new' and mess_check > time):
            mail_check[pos] = 'update'
            
        ID = message['data']['Message-ID']
        try:
            message['data']['References']                           
            if((not thread == None) and (thread['data']['Message-ID'] not in message['data']['References'])):
                di[k] = thread
                store = None
                k+=1
                
            elif( (not store == None) and (store['data']['Message-ID'] not in message['data']['References'])):
                di[k] = store
                store = None
                k+=1
            thread = message
        except:
            if(not thread == None):
                di[k] = thread
                thread = None
                k+=1
            elif(not store == None):
                di[k] = store
                store = None
                k+=1
            store = message
        if(len(di) == 5000):
            print("Before")
            numb,mail_lists,mail_check = Piper.make(table,db,mail_check,archives,mail_lists,res,session,di,numb)
            print("Hey")
            di = {}
            k = 0
    if(len(di) < 5000 and len(di) > 0):
        print("right after")
        print(len(di))
        numb,mail_lists,mail_check = Piper.make(table,db,mail_check,archives,mail_lists,res,session,di,numb)
        di = {}
        k = 0
    else:
        di = {}
        k = 0
    if( (thread == None) and (store == None)):
        good = 1
    elif( (thread == None) and (not store == None) ):
        di[k] = store
    elif( (store == None) and (not thread == None)):
        di[k] = thread
    elif(store['data']['Message-ID'] in thread['data']['References']):
        di[k] = thread
    else:
        di[k] = store
    if(bool(di)):
        print("outside")
        numb,mail_lists,mail_check = Piper.make(table,db,mail_check,archives,mail_lists,res,session,di,numb)
    return numb,mail_lists
    

## Iteration through mailing lists:

Determines if the file with the mailing lists was created, if not it writes a set of default mailing lists (to show how the program would work). The mailing lists are then loaded into a dataframe and we iterate through the mailing lists by grouping them by the links. We then check to see if the mailing list is in the 'mailing_list_jobs' table in the SQL Database and if so we assign 'not_new' to 'mail_check' and store the last message date that is stored in 'mailing_list_jobs' to 'time'. If the mailing list is not in 'mailing_list_jobs' we assign 'new' to 'mail_check' and the date is set to 'None' for 'time'.

In [5]:
if(not os.path.exists(path)):
    file = open(path, "w+")
else:
    file = open(path, "r")
    print("yeah")
if (os.stat(path).st_size == 0):
    file.write("Link,mail_list\n")
    file.write("https://lists.opendaylight.org/pipermail/,\"aalldp-dev\"\n")
    file.write("https://lists.opendaylight.org/pipermail/,\"archetypes-dev\"\n")
    print("Please enter the mailing lists and the links for them please")
    print("Going to the default mailing lists")
    file.close()
    file = open(path,"r")

count = 0
for line in file:
    print(line)
    count+=1
    if(count == 2):
        break
file.close()
if(count == 2):
    df = pd.read_csv(path)
    groups = df.groupby('Link').groups
    for group in groups:
        link = group
        mail_list = (df.loc[df['Link'] == group]['mail_list']).tolist()
        print(mail_list,"mail_list")            
        mail_check = {key:False for key in mail_list}
        print(mail_check)
        file = "mail_list"
        path = "/augur/data/archive-" 
        #numb = 0
        for x in range(len(mail_list)):
            #print(link+mail[x])
            if(mail_list[x] not in df1['project'].values ):
                mail_check[mail_list[x]] = 'new'
                place = os.path.join(os.getcwd() + path + mail_list[x] +'.json')           
                repo = Pipermail(url = link+ mail_list[x] + "/",dirpath="tmp/archives_"+mail_list[x])
                numb,mail_lists = write_message(table,repo,'new',mail_check,mail_list[x],db,res,session,\
                                     [mail_list[x]],numb,mail_lists)
                print("Created File",mail_list[x])
            else:
                last_updatedSQL = s.sql.text("""SELECT last_message_date FROM 
                mailing_list_jobs WHERE project = """ +  "'" + mail_list[x] + "'")
                last_updated_df = pd.read_sql(last_updatedSQL, db)
                time = (last_updated_df['last_message_date'])  
                time = time.astype(object)
                place = os.path.join(os.getcwd() + path + 'temp_' + mail_list[x] +'.json')       
                repo = Pipermail(url = link+ mail_list[x] + "/",dirpath="tmp/archives_"+mail_list[x])
                print(time[0])
                print(type(time[0]))
                numb,mail_lists = write_message(table,repo,'not_new',mail_check,mail_list[x],db,\
                              res,session,[mail_list[x]],numb,mail_lists,time[0])
                print("Checking to see for updated messages")
        print(mail_check)
        print("Finished downloading files")

2018-10-01 21:58:47 keanu-Inspiron-5567 perceval.backends.core.pipermail[25251] INFO Looking for messages from 'https://lists.gluster.org/pipermail/gluster-users/' since 2010-12-01 20:37:11
2018-10-01 21:58:47 keanu-Inspiron-5567 perceval.backends.core.pipermail[25251] INFO Downloading mboxes from 'https://lists.gluster.org/pipermail/gluster-users/' to since 2010-12-01 20:37:11


yeah
Link,mail_list

https://lists.gluster.org/pipermail/,"gluster-users"

['gluster-users', 'announce', 'gluster-devel', 'gluster-infra'] mail_list
{'gluster-users': False, 'announce': False, 'gluster-devel': False, 'gluster-infra': False}


2018-10-01 22:01:19 keanu-Inspiron-5567 perceval.backends.core.pipermail[25251] INFO 95/126 MBoxes downloaded
2018-10-01 22:01:19 keanu-Inspiron-5567 perceval.backends.core.mbox[25251] WARNING Field 'Message-ID' not found in message the finder when trying to copy a file to the share (CIFS) I get an error that says "The Finder can't complete the operation because some data is "export-20100324(2).tar.bz can't be read or written. (Error code -36)". Additionally when I try and copy a file it will create the stub of the file, but wont actually copy the data.; ignoring
2018-10-01 22:01:19 keanu-Inspiron-5567 perceval.backends.core.mbox[25251] WARNING Field 'Message-ID' not found in message the Finder when I do a "Get Info" to look at the sharing & permissions settings it states that I have unknown access. ; ignoring
2018-10-01 22:01:19 keanu-Inspiron-5567 perceval.backends.core.mbox[25251] WARNING Field 'Message-ID' not found in message the finder and from the command line I am able to creat

Before
New value:  True
['gluster-users']
Length:  5000
Higher  0
Higher  500
Higher  1000
Higher  1500
Should not be here
Higher  2000
Higher  2500
Higher  3000
Should not be here
Higher  3500
Higher  4000
Higher  4500
Should not be here
lower and lower
right here
2013-03-06 00:00:35
  augurlistID backend_name                                   mailing_list_url  \
0           1    Pipermail  https://lists.gluster.org/pipermail/gluster-us...   

         project   last_message_date  
0  gluster-users 2018-10-02 02:01:25  
File uploaded  16068
Mailing List Job uploaded
Finished
Hey


2018-10-01 22:03:41 keanu-Inspiron-5567 perceval.backends.core.mbox[25251] WARNING Field 'Message-ID' not found in message the other node,; ignoring
2018-10-01 22:03:41 keanu-Inspiron-5567 perceval.backends.core.mbox[25251] WARNING Field 'Message-ID' not found in message fw1:; ignoring
2018-10-01 22:03:41 keanu-Inspiron-5567 perceval.backends.core.mbox[25251] WARNING Field 'Message-ID' not found in message Vijay:; ignoring
2018-10-01 22:03:41 keanu-Inspiron-5567 perceval.backends.core.mbox[25251] WARNING Field 'Message-ID' not found in message existing nodes trying to do:; ignoring
2018-10-01 22:03:41 keanu-Inspiron-5567 perceval.backends.core.mbox[25251] WARNING Field 'Message-ID' not found in message existing nodes I see operating-version=1; ignoring
2018-10-01 22:03:41 keanu-Inspiron-5567 perceval.backends.core.mbox[25251] WARNING Field 'Message-ID' not found in message existing nodes trying to do:; ignoring
2018-10-01 22:03:41 keanu-Inspiron-5567 perceval.backends.core.mbox[25251] 

...

right after
3859
New value:  True
['gluster-infra']
Length:  3859
Higher  0
Higher  500
Higher  1000
Higher  1500
Should not be here
Higher  2000
Higher  2500
Higher  3000
Should not be here
Higher  3500
lower and lower
right here
2018-10-01 07:37:13
  augurlistID backend_name                                   mailing_list_url  \
0           4    Pipermail  https://lists.gluster.org/pipermail/gluster-in...   

         project   last_message_date  
0  gluster-infra 2018-10-02 02:27:17  
File uploaded  11792
Mailing List Job uploaded
Finished
outside
New value:  False
['gluster-infra']
Length:  1
Higher  0
lower and lower
right here


2018-10-01 22:28:33 keanu-Inspiron-5567 perceval.backends.core.pipermail[25251] INFO Looking for messages from 'https://lists.opendaylight.org/pipermail/aalldp-dev/' since 2010-12-01 20:37:11
2018-10-01 22:28:33 keanu-Inspiron-5567 perceval.backends.core.pipermail[25251] INFO Downloading mboxes from 'https://lists.opendaylight.org/pipermail/aalldp-dev/' to since 2010-12-01 20:37:11


File uploaded  4
Finished
Created File gluster-infra
{'gluster-users': 'update', 'announce': 'update', 'gluster-devel': 'update', 'gluster-infra': 'update'}
Finished downloading files
['aalldp-dev'] mail_list
{'aalldp-dev': False}


2018-10-01 22:28:37 keanu-Inspiron-5567 perceval.backends.core.pipermail[25251] INFO 2/2 MBoxes downloaded
2018-10-01 22:28:37 keanu-Inspiron-5567 perceval.backends.core.mbox[25251] INFO Done. 6/6 messages fetched; 0 ignored
2018-10-01 22:28:37 keanu-Inspiron-5567 perceval.backends.core.pipermail[25251] INFO Fetch process completed


right after
4
New value:  True
['aalldp-dev']
Length:  4
Higher  0
lower and lower
right here
2018-07-06 00:00:58
  augurlistID backend_name                                   mailing_list_url  \
0           5    Pipermail  https://lists.opendaylight.org/pipermail/aalld...   

      project   last_message_date  
0  aalldp-dev 2018-10-02 02:28:37  
File uploaded  13
Mailing List Job uploaded
Finished
outside
New value:  False
['aalldp-dev']
Length:  1
Higher  0
lower and lower
right here
File uploaded  4
Finished
Created File aalldp-dev
{'aalldp-dev': 'update'}
Finished downloading files


In [6]:
li = ['Thu,', '2', 'Aug', '2012', '13:28:10', '-0700']

In [7]:
li1 = ['Thu,', '6', 'Jan', '2011', '17:26:57', '-0500', '(EST)']

In [8]:
li1[4]

'17:26:57'

In [9]:
("Tue, 11 Feb 2014 15:18:55").split()

['Tue,', '11', 'Feb', '2014', '15:18:55']

In [10]:
("15:18:55").split(":")

['15', '18', '55']